# Project 1 - Search problem
---

Using AIMA library (https://github.com/aimacode/aima-python/blob/master/search.ipynb), we need to to move the robot from initial position to final position, without collide with the wall.  

The environment is known and deterministic.  

Robot position (x, y)

In [1]:
from search import *
from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

import numpy as np
import os
from used_search import *

In [3]:
if not os.path.exists('img'):
    os.makedirs('img')

# Modeling the env
---
0 < x < 60  
0 < y < 60

up:    state[1] + step  
down:  state[1] - step  
left:  state[0] - step  
right: state[0] + step

In [4]:
roam_globalset = set()

In [5]:
class SimpleSearchProblem(Problem):
    def __init__(self, step=1):
        self.initial = (10,10)
        self.goal = (50,50)
        self.step = step
#         self.explored = set()
        
        
    def actions(self, state):
        # save expanded nodes
        roam_globalset.add(state)
        
        up = state[1] + self.step
        down = state[1] - self.step
        left = state[0] - self.step
        right = state[0] + self.step
        
        # can go only up
        if (state[0] < 20 and state[1] <= 40 and down <= 0 and left <= 0 and right >= 20):
            return iter([(state[0], up)])
        # can go only down
        elif (state[0] > 40 and state[1] >= 20 and up >= 60 and left <= 40 and right >= 60):
            return iter([(state[0], down)])
        
        # can go up and down
        elif (state[0] < 20 and state[1] <= 40 and left <= 0 and right >= 20) or \
                (20 < state[0] and state[0] < 40 and state[1] >= 20 and state[1] <= 40 and left <= 20 and right >= 40) or \
                 (state[0] > 40 and state[1] > 20 and left <= 40 and right >= 60):
            return iter([(state[0], up), (state[0], down)])
        # can go up and left
        elif (down <= 0 and right >= 60) or \
                (state[0] < 20 and down <= 0 and right >= 20):
            return iter([(state[0], up), (left, state[1])])
        # can go up and right
        elif (down <= 0 and left <= 0) or (state[0] > 20 and down <= 0 and right <= 20):
            return iter([(state[0], up), (right, state[1])])
        # can go left and right
        elif (state[0] == 20 and state[1] > 40) or (state[0] == 40 and state[1] < 20):
            return iter([(left, state[1]), (right, state[1])])
        # can go left and down
        elif (up >= 60 and right >= 60) or \
                (state[0] < 40 and state[1] >= 20 and right >= 40):
            return iter([(left, state[1]), (state[0], down)])
        # can go right and down
        elif (up >= 60 and left <= 0) or \
                (state[0] > 40 and state[1] >= 20 and left <= 20):
            return iter([(right, state[1]), (state[0], down)])
        
        # _dlr
        elif up >= 60:
            return iter([(state[0], down), (left, state[1]), (right, state[1])])
        # u_lr
        elif down <= 0:
            return iter([(state[0], up), (left, state[1]), (right, state[1])])
        # ud_r
        elif (left <= 0) or \
                (state[0] > 20 and state[1] >= 20 and left <= 40) or \
                (state[0] > 20 and state[0] < 40 and state[1] <= 40 and left <= 20):
            return iter([(state[0], up), (state[0], down), (right, state[1])])
        # udl_
        elif (right >= 60) or \
                (state[0] < 20 and state[1] <= 40 and right >= 20) or \
                (state[0] > 20 and state[0] < 40 and state[1] >= 20 and right >= 40):
            return iter([(state[0], up), (state[0], down), (left, state[1])])
        # no restriction
        else:
#             print('NOOOOOOOOO')
            return iter([(state[0], up), (state[0], down), (left, state[1]), \
                   (right, state[1])])
        
    def result(self, state, action):
        return action

    def path_cost(self, c, state1, action, state2):
        return c+1

    def goal_test(self, state):
        if state == (50,50):
            return True
        else:
            return False        

## Uninformed Search Strategies

### 1) Uniform Cost Search

In [6]:
for i in [1, 2, 5, 10]:
    roam_globalset.clear()
    
    search_problem = SimpleSearchProblem(i)

    ucs = uniform_cost_search(search_problem).solution()

    color = '#aaaaaa'
    linewidth = 4
    plt.plot(np.linspace(0,60,200), np.zeros(200), color, linewidth=linewidth)
    plt.plot(np.linspace(0,60,200), np.ones(200)*60, color, linewidth=linewidth)
    plt.plot(np.zeros(200), np.linspace(0,60,200) , color ,linewidth=linewidth)
    plt.plot(np.ones(200)*60, np.linspace(0,60,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*40, np.linspace(60,20,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*20, np.linspace(0,40,200) , color, linewidth=linewidth)

    plt.scatter(10,10, marker='x', color='g', s=200)
    plt.scatter(50,50, marker='*', color='r', s=200)
    plt.title('Uniform Cost Search')
    plt.scatter(*zip(*ucs))
    plt.scatter(*zip(*roam_globalset), alpha=0.1, color='g')
    plt.savefig('img/ucs_' + str(i) + '.png')
    plt.close()

### 2) Depth First Search

In [7]:
for i in [1, 2, 5, 10]:
    
    roam_globalset.clear()
    search_problem = SimpleSearchProblem(i)

    dfs = depth_first_graph_search(search_problem).solution()

    color = '#aaaaaa'
    linewidth = 4
    plt.plot(np.linspace(0,60,200), np.zeros(200), color, linewidth=linewidth)
    plt.plot(np.linspace(0,60,200), np.ones(200)*60, color, linewidth=linewidth)
    plt.plot(np.zeros(200), np.linspace(0,60,200) , color ,linewidth=linewidth)
    plt.plot(np.ones(200)*60, np.linspace(0,60,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*40, np.linspace(60,20,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*20, np.linspace(0,40,200) , color, linewidth=linewidth)

    plt.scatter(10,10, marker='x', color='g', s=200)
    plt.scatter(50,50, marker='*', color='r', s=200)
    plt.title('Depth First Search')
    plt.scatter(*zip(*dfs))
    plt.scatter(*zip(*roam_globalset), alpha=0.1, color='g')
    plt.savefig('img/dfs_' + str(i) + '.png')
    plt.close()


## Informed (Heuristic) Search Strategies

### 1) A* linear

In [8]:
for i in [1, 2, 5, 10]:
    
    roam_globalset.clear()
    search_problem = SimpleSearchProblem(i)

    astar = astar_search(search_problem, linear).solution()

    color = '#aaaaaa'
    linewidth = 4
    plt.plot(np.linspace(0,60,200), np.zeros(200), color, linewidth=linewidth)
    plt.plot(np.linspace(0,60,200), np.ones(200)*60, color, linewidth=linewidth)
    plt.plot(np.zeros(200), np.linspace(0,60,200) , color ,linewidth=linewidth)
    plt.plot(np.ones(200)*60, np.linspace(0,60,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*40, np.linspace(60,20,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*20, np.linspace(0,40,200) , color, linewidth=linewidth)

    plt.scatter(10,10, marker='x', color='g', s=200)
    plt.scatter(50,50, marker='*', color='r', s=200)
    plt.title('A* Search')
    plt.scatter(*zip(*astar))
    plt.scatter(*zip(*roam_globalset), alpha=0.1, color='g')
    plt.savefig('img/astar_' + str(i) + '.png')
    plt.close()


### 2) A* manhattan

In [9]:
for i in [1, 2, 5, 10]:
    
    roam_globalset.clear()
    search_problem = SimpleSearchProblem(i)

    astar = astar_search(search_problem, manhattan).solution()

    color = '#aaaaaa'
    linewidth = 4
    plt.plot(np.linspace(0,60,200), np.zeros(200), color, linewidth=linewidth)
    plt.plot(np.linspace(0,60,200), np.ones(200)*60, color, linewidth=linewidth)
    plt.plot(np.zeros(200), np.linspace(0,60,200) , color ,linewidth=linewidth)
    plt.plot(np.ones(200)*60, np.linspace(0,60,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*40, np.linspace(60,20,200) , color, linewidth=linewidth)
    plt.plot(np.ones(200)*20, np.linspace(0,40,200) , color, linewidth=linewidth)

    plt.scatter(10,10, marker='x', color='g', s=200)
    plt.scatter(50,50, marker='*', color='r', s=200)
    plt.title('A* Search')
    plt.scatter(*zip(*astar))
    plt.scatter(*zip(*roam_globalset), alpha=0.1, color='g')
    plt.savefig('img/astar_man' + str(i) + '.png')
    plt.close()


# %%timeit

### step = 0.5

In [ ]:
search_problem = SimpleSearchProblem(0.5)

In [ ]:
%%timeit -n1
uniform_cost_search(search_problem)

In [ ]:
%%timeit -n1
depth_first_graph_search(search_problem)

In [ ]:
%%timeit -n1
astar_search(search_problem, linear)

In [ ]:
%%timeit -n1
astar_search(search_problem, manhattan)

### step = 1

In [10]:
search_problem = SimpleSearchProblem(1)

In [11]:
%%timeit -n1
uniform_cost_search(search_problem)

198 ms ± 8.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit -n1
depth_first_graph_search(search_problem)

779 ms ± 56.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit -n1
astar_search(search_problem, linear)

255 ms ± 42.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit -n1
astar_search(search_problem, manhattan)

287 ms ± 9.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### step = 2

In [15]:
search_problem = SimpleSearchProblem(2)

In [16]:
%%timeit -n10
uniform_cost_search(search_problem)

24.6 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit -n10
depth_first_graph_search(search_problem)

28.9 ms ± 695 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit -n10
astar_search(search_problem, linear)

29 ms ± 820 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit -n10
astar_search(search_problem, manhattan)

32.6 ms ± 858 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### step = 5

In [20]:
search_problem = SimpleSearchProblem(5)

In [21]:
%%timeit -n1000
uniform_cost_search(search_problem)

1.94 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
%%timeit -n1000
depth_first_graph_search(search_problem)

1.29 ms ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%%timeit -n1000
astar_search(search_problem, linear)

1.95 ms ± 75.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%%timeit -n1000
astar_search(search_problem, manhattan)

2.06 ms ± 67.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Generate some images for the report

In [25]:
color = '#aaaaaa'
linewidth = 4
plt.plot(np.linspace(0,60,200), np.zeros(200), color, linewidth=linewidth)
plt.plot(np.linspace(0,60,200), np.ones(200)*60, color, linewidth=linewidth)
plt.plot(np.zeros(200), np.linspace(0,60,200) , color ,linewidth=linewidth)
plt.plot(np.ones(200)*60, np.linspace(0,60,200) , color, linewidth=linewidth)
plt.plot(np.ones(200)*40, np.linspace(60,20,200) , color, linewidth=linewidth)
plt.plot(np.ones(200)*20, np.linspace(0,40,200) , color, linewidth=linewidth)

plt.scatter(10,10, marker='x', color='g', s=200)
plt.scatter(50,50, marker='*', color='r', s=200)
plt.title('Map')
plt.savefig('img/map.png')
plt.close()
